In [7]:
import os
import ipynbname
if 'VSCODE_PID' in os.environ:
    print("Notebook exécuté dans VSCode.")
    repertoire_racine_git_local = (os.getcwd()).replace("\\notebooks", "")
else:
    print("Notebook exécuté dans Jupyter Notebook/JupyterLab.")
    #notebook_filename = ipynbname.name() # récupération le nom du fichier du notebook
    notebook_filepath = ipynbname.path() # récupération du chemin complet du notebook
    # Remplacement des backslashes par des slashes, et remontée d'un niveau avec \"/../\"
    repertoire_racine_git_local = (os.path.dirname(notebook_filepath).replace("\\notebooks", "").replace('\\', '/'))
    
print("Répertoire racine du repo Git en local :", repertoire_racine_git_local)

Notebook exécuté dans VSCode.
Répertoire racine du repo Git en local : z:\home\ejaffuel\SEP24-MLOPS-SAKE_Linux


In [8]:
# pour permettre les imports de package depuis "steps.xxx" (pour les notebooks Jupyter ou VS Code)
import sys
sys.path.append(repertoire_racine_git_local) 

In [9]:
# Import des librairies
import requests
import pandas as pd
import numpy as np
import os
from typing import List
from steps.ingest import get_full_text, get_new_images_to_ocerize, get_processed_dataset, save_text_to_file
ocr_endpoint = "http://localhost:8901/txt/blocks-words" # url de l'OCR
data_path = repertoire_racine_git_local + "/data/" # chemin du dossier data

In [11]:
# Appel à l'OCR (TEST)
test_image_path = repertoire_racine_git_local + "/data/raw/final/img_0000000.jpg" 
result = get_full_text(test_image_path, ocr_endpoint)
print(result)

FACTURE
LOGO
Joanna Binet 48 Coubertin 31400 Paris FACTURÉ À Cendrillon Ayot 69 rue Nations 22000 Paris
ENVOYÉA Cendrillon Ayot 46 Rue St Ferréol 92360 ile-de-France
FACTURE No DATE COMMANDE N ÉCHÉANCE PRIX UNIT. HT 100.00 15.00 5.00 Total HT TVA 20.0% TOTAL
FR-001 29/01/2019 1630/2019 24/05/2019 MONTANT HT 100.00 30.00 15.00 145.00 29.00 174.00 e
QTÉ 1 2 3
DÉSIGNATION
Grand brun escargot pour manger Petit marinière uniforme en bleu Facile à jouer accordéon
- - - M -
CONDITIONS ET MODALITÉS DE PAIEMENT Le paiement est do dans 15 jours Caisse d'Epargne IBAN: FR12 1234 5678 SWIFT/BIC: ABCDFRP1XXX
Mlesci


In [12]:
# Appel au clean
clean_url = "http://localhost:8903/clean" # url du cleaner/tokenizer
def get_cleaned_text(text) -> str:
    print(clean_url)
    headers = {'Content-Type': 'text/plain'}
    params = {
        "text": text
    }
    response = requests.post(clean_url, params=params, headers=headers)
    return response
get_cleaned_text(result).text

http://localhost:8903/clean


'factur logo joanna binet 48 coubertin 31400 pari facturer cendrillon ayot 69 rue nation 22000 envoyéa 46 st ferréol 92360 ile-de-france no date command échéance prix unir ht 100.00 15.00 5.00 total tva 20.0 fr-001 29/01/2019 1630/2019 24/05/2019 montant 30.00 145.00 29.00 174.00 qté désignation grand brun escargot manger petit marinier uniforme bleu facile jouer accordéon condition modalité paiement do 15 jour caisse epargne iban fr12 1234 5678 swift / bic abcdfrp1xx mlesci'

In [6]:
# TEST INGEST EN MASSE
path_to_dataset = f"{data_path}/processed/processed_dataset.csv"
raw_dataset = pd.read_csv(f"{data_path}working_dataset.csv")
processed_dataset = get_processed_dataset(path_to_dataset)

new_images = get_new_images_to_ocerize(raw_dataset, processed_dataset)

for _, row in new_images.iterrows():                                               # TODO : Attention j'ai mis head(10) pour tester, à enlever
    full_text = get_full_text(f"{data_path}raw/final/{row.filename}", ocr_endpoint)

    text_file_path = f"{data_path}/processed/{row.filename}.txt"
    save_text_to_file(full_text, text_file_path)
    new_images.loc[new_images['filename']==row.filename, 'full_text'] = text_file_path 

processed_dataset = pd.concat([processed_dataset, new_images],ignore_index=True)   # TODO : Attention j'ai mis head(10) pour tester, à enlever
processed_dataset.to_csv(path_to_dataset, index=None)

C:\Users\Eddie\AppData\Local\Temp\ipykernel_23224\1777794942.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_images.loc[new_images['filename']==row.filename, 'full_text'] = text_file_path


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))